* [issues/307](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/307)

In [7]:
import requests
import pandas as pd

def build_image_url(media_id, size="400x300"):
    """
    Försöker hitta rätt domän för bilden: mm, ems, am, mueum etc.
    Testar i ordning och returnerar första fungerande URL.
    """
    if not media_id:
        return None
    
    domains = ["mm", "ems", "am", "mueum"]
    for d in domains:
        url = f"https://{d}.dimu.org/image/{media_id}?dimension={size}"
        try:
            r = requests.head(url, timeout=2)
            if r.status_code == 200:
                return url
        except requests.RequestException:
            continue
    return None


def search_digitaltmuseum_df(query, rows=10):
    url = "https://api.dimu.org/api/solr/select"
    params = {
        "q": query,
        "wt": "json",
        "rows": rows,
        "api.key": "demo"
    }

    r = requests.get(url, params=params)
    if not r.ok:
        raise RuntimeError(f"Fel {r.status_code}: {r.text}")

    data = r.json()
    response = data.get("response", {})
    docs = response.get("docs", [])
    num_found = response.get("numFound", 0)

    print(f"🔎 {num_found} träffar för '{query}'\n")

    rows_data = []
    for doc in docs:
        title = doc.get("artifact.ingress.title", "(utan titel)")
        desc = doc.get("artifact.ingress.description", "")
        producer = doc.get("artifact.ingress.producer", "")
        role = doc.get("artifact.ingress.producerRole", "")
        license_info = ", ".join(doc.get("artifact.ingress.license", []))
        unique_id = doc.get("artifact.uniqueId", "")
        img_id = doc.get("artifact.defaultMediaIdentifier")
        img_url = build_image_url(img_id)
        web_url = f"https://digitaltmuseum.se/{unique_id}" if unique_id else ""

        rows_data.append({
            "Titel": title,
            "Beskrivning": desc,
            "Fotograf": f"{producer} ({role})" if producer else "",
            "Licens": license_info,
            "DigitaltMuseum": web_url,
            "Bild": img_url
        })

    df = pd.DataFrame(rows_data)
    return df

# Exempel: sök efter Sandhamn-relaterade objekt
df = search_digitaltmuseum_df("Sandhamn", rows=10)

# Visa klickbara länkar och bilder i Jupyter
from IPython.display import HTML
def make_clickable(val):
    return f'<a href="{val}" target="_blank">{val}</a>' if val else ""

def make_img(val):
    return f'<img src="{val}" width="160"/>' if val else ""

df_html = df.copy()
df_html["DigitaltMuseum"] = df_html["DigitaltMuseum"].apply(make_clickable)
df_html["Bild"] = df_html["Bild"].apply(make_img)

HTML(df_html.to_html(escape=False, index=False))


🔎 2541 träffar för 'Sandhamn'



Titel,Beskrivning,Fotograf,Licens,DigitaltMuseum,Bild
Namnbräda,"Namnbräda till vedettbåten Sandhamn. Rektangulär fernissad träplatta med bokstäver av stål bildande namnet ""Sandhamn"". Bokstäverna skruvade i träplattan.",,CC by,https://digitaltmuseum.se/011024821686,
Sandhamn fiskeläge,"Sandhamn fiskeläge, Fröjel socken",,,https://digitaltmuseum.se/0210213568952,
"Extreriör. Fiskestugor, Sandhamn.",,okänd (Fotograf),CC by,https://digitaltmuseum.se/021017826085,
SANDHAMN (1984),,,,https://digitaltmuseum.se/021176011099,
(utan titel),,"Lindblom, Thure (Fotograf)",,https://digitaltmuseum.se/021015769722,
(utan titel),,Okänd fotograf (Fotograf),CC by-sa,https://digitaltmuseum.se/021016103502,
Sandhamn,,"Nordström, Alf (Fotograf)",CC by,https://digitaltmuseum.se/0210114021447,
Sandhamn,,"Nordström, Alf (Fotograf)",CC by,https://digitaltmuseum.se/0210114021448,
Sandhamn,,"Lundkvist, Ingvar (Fotograf)",CC by,https://digitaltmuseum.se/0210114040518,
(utan titel),,Okänd fotograf (Fotograf),CC pdm,https://digitaltmuseum.se/021015579958,


In [8]:
df["Bild"].head()

0    https://mm.dimu.org/image/042uKXJQ6XRz?dimensi...
1                                                 None
2    https://mm.dimu.org/image/0231wTu6JqxB?dimensi...
3                                                 None
4    https://mm.dimu.org/image/012uP2GRP7pj?dimensi...
Name: Bild, dtype: object

In [9]:
from IPython.display import display, HTML

def make_img(val):
    return f'<img src="{val}" width="150">' if val else ""

df_html = df.copy()
df_html["DigitaltMuseum"] = df_html["DigitaltMuseum"].apply(lambda x: f'<a href="{x}" target="_blank">🔗</a>')
df_html["Bild"] = df_html["Bild"].apply(make_img)

display(HTML(df_html.to_html(escape=False, index=False)))


Titel,Beskrivning,Fotograf,Licens,DigitaltMuseum,Bild
Namnbräda,"Namnbräda till vedettbåten Sandhamn. Rektangulär fernissad träplatta med bokstäver av stål bildande namnet ""Sandhamn"". Bokstäverna skruvade i träplattan.",,CC by,🔗,
Sandhamn fiskeläge,"Sandhamn fiskeläge, Fröjel socken",,,🔗,
"Extreriör. Fiskestugor, Sandhamn.",,okänd (Fotograf),CC by,🔗,
SANDHAMN (1984),,,,🔗,
(utan titel),,"Lindblom, Thure (Fotograf)",,🔗,
(utan titel),,Okänd fotograf (Fotograf),CC by-sa,🔗,
Sandhamn,,"Nordström, Alf (Fotograf)",CC by,🔗,
Sandhamn,,"Nordström, Alf (Fotograf)",CC by,🔗,
Sandhamn,,"Lundkvist, Ingvar (Fotograf)",CC by,🔗,
(utan titel),,Okänd fotograf (Fotograf),CC pdm,🔗,


In [10]:
from IPython.display import Image
Image(url="https://ems.dimu.org/image/042uKXJQ6XRz?dimension=400x300")


In [11]:
from IPython.display import HTML

def show_df_with_images(df):
    # kopiera för att inte modifiera originalet
    df_html = df.copy()

    # skapa klickbar länk till DigitaltMuseum
    df_html["DigitaltMuseum"] = df_html["DigitaltMuseum"].apply(
        lambda x: f'<a href="{x}" target="_blank">🔗</a>' if x else ""
    )

    # bädda in bilder direkt i HTML
    df_html["Bild"] = df_html["Bild"].apply(
        lambda x: f'<img src="{x}" width="160"/>' if x else ""
    )

    # rendera som HTML
    return HTML(df_html.to_html(escape=False, index=False))

# Visa tabellen med inlinebilder
show_df_with_images(df)


Titel,Beskrivning,Fotograf,Licens,DigitaltMuseum,Bild
Namnbräda,"Namnbräda till vedettbåten Sandhamn. Rektangulär fernissad träplatta med bokstäver av stål bildande namnet ""Sandhamn"". Bokstäverna skruvade i träplattan.",,CC by,🔗,
Sandhamn fiskeläge,"Sandhamn fiskeläge, Fröjel socken",,,🔗,
"Extreriör. Fiskestugor, Sandhamn.",,okänd (Fotograf),CC by,🔗,
SANDHAMN (1984),,,,🔗,
(utan titel),,"Lindblom, Thure (Fotograf)",,🔗,
(utan titel),,Okänd fotograf (Fotograf),CC by-sa,🔗,
Sandhamn,,"Nordström, Alf (Fotograf)",CC by,🔗,
Sandhamn,,"Nordström, Alf (Fotograf)",CC by,🔗,
Sandhamn,,"Lundkvist, Ingvar (Fotograf)",CC by,🔗,
(utan titel),,Okänd fotograf (Fotograf),CC pdm,🔗,


In [12]:
with open("digitaltmuseum_sandhamn.html", "w") as f:
    f.write(show_df_with_images(df).data)


In [13]:
import requests
import pandas as pd
from IPython.display import HTML

def build_image_url(media_id, size="400x300"):
    """Returnerar fungerande bild-URL genom att testa flera domäner."""
    if not media_id:
        return None
    # ordning: nya servrar först
    domains = ["ems", "mm", "am", "mueum", "musit"]
    for d in domains:
        url = f"https://{d}.dimu.org/image/{media_id}?dimension={size}"
        try:
            r = requests.head(url, timeout=2)
            if r.status_code == 200:
                return url
        except requests.RequestException:
            pass
    return None


def search_digitaltmuseum_df(query, rows=10):
    """Sök i DigitaltMuseum och returnera DataFrame med titel, bild, m.m."""
    url = "https://api.dimu.org/api/solr/select"
    params = {
        "q": query,
        "wt": "json",
        "rows": rows,
        "api.key": "demo"
    }
    r = requests.get(url, params=params)
    if not r.ok:
        raise RuntimeError(f"Fel {r.status_code}: {r.text}")
    data = r.json()
    response = data.get("response", {})
    docs = response.get("docs", [])
    num_found = response.get("numFound", 0)
    print(f"🔎 {num_found} träffar för '{query}'\n")

    rows_data = []
    for doc in docs:
        title = doc.get("artifact.ingress.title", "(utan titel)")
        desc = doc.get("artifact.ingress.description", "")
        producer = doc.get("artifact.ingress.producer", "")
        role = doc.get("artifact.ingress.producerRole", "")
        license_info = ", ".join(doc.get("artifact.ingress.license", []))
        unique_id = doc.get("artifact.uniqueId", "")
        img_id = doc.get("artifact.defaultMediaIdentifier")
        img_url = build_image_url(img_id)
        web_url = f"https://digitaltmuseum.se/{unique_id}" if unique_id else ""
        rows_data.append({
            "Titel": title,
            "Beskrivning": desc,
            "Fotograf": f"{producer} ({role})" if producer else "",
            "Licens": license_info,
            "DigitaltMuseum": web_url,
            "Bild": img_url
        })

    return pd.DataFrame(rows_data)


def show_df_with_images(df):
    """Rendera DataFrame med klickbara bilder i Jupyter."""
    df_html = df.copy()
    df_html["DigitaltMuseum"] = df_html["DigitaltMuseum"].apply(
        lambda x: f'<a href="{x}" target="_blank">🔗</a>' if x else ""
    )
    df_html["Bild"] = df_html["Bild"].apply(
        lambda x: f'<img src="{x}" width="160"/>' if x else ""
    )
    return HTML(df_html.to_html(escape=False, index=False))


# Testa
df = search_digitaltmuseum_df("Sandhamn", rows=10)
show_df_with_images(df)


🔎 2541 träffar för 'Sandhamn'



Titel,Beskrivning,Fotograf,Licens,DigitaltMuseum,Bild
Namnbräda,"Namnbräda till vedettbåten Sandhamn. Rektangulär fernissad träplatta med bokstäver av stål bildande namnet ""Sandhamn"". Bokstäverna skruvade i träplattan.",,CC by,🔗,
Sandhamn fiskeläge,"Sandhamn fiskeläge, Fröjel socken",,,🔗,
"Extreriör. Fiskestugor, Sandhamn.",,okänd (Fotograf),CC by,🔗,
SANDHAMN (1984),,,,🔗,
(utan titel),,"Lindblom, Thure (Fotograf)",,🔗,
(utan titel),,Okänd fotograf (Fotograf),CC by-sa,🔗,
Sandhamn,,"Nordström, Alf (Fotograf)",CC by,🔗,
Sandhamn,,"Nordström, Alf (Fotograf)",CC by,🔗,
Sandhamn,,"Lundkvist, Ingvar (Fotograf)",CC by,🔗,
(utan titel),,Okänd fotograf (Fotograf),CC pdm,🔗,
